# GROMACS Resume -  Start a long GROMACS run that times out, and resume it

On the supercomputers, a maximum runtime of 24h is enforced. If a longer GROMACS run is needed, the run can easily be resumed. One of the outputs provided by the Rush `gmx` module is designed to be used as the input to the `gmx_resume` module, which will resume the run from the latest checkpoint. The outputs to this module are identical to those of the `gmx` module itself, so the run can be resumed as many times as necessary to finish it.

## 0.0) Imports

In [ ]:
from pathlib import Path
import rush

In [ ]:
# |hide
# Users won't generally create a workspace
# We nuke to ensure run is reproducible
import os

WORK_DIR = Path.home() / "qdx" / "tutorial-gmx-resume"

if WORK_DIR.exists():
    client = rush.Provider(workspace=WORK_DIR)
    await client.nuke(remote=False)

os.makedirs(WORK_DIR, exist_ok=True)

2024-03-22 16:01:05,862 - rush - INFO - Restoring by default via env


In [ ]:
RUSH_TOKEN = os.getenv("RUSH_TOKEN") or "YOUR_TOKEN_HERE"
client = rush.build_blocking_provider_with_functions(access_token=RUSH_TOKEN)

2024-03-22 16:01:05,887 - rush - INFO - Restoring by default via env


In [ ]:
# |hide
# We hide this because users will generally not set a workspace, and won't restore by default
client = rush.build_blocking_provider_with_functions(
    batch_tags=["tutorial-resume-gmx"],
    workspace=WORK_DIR,
)

2024-03-22 16:01:07,481 - rush - INFO - Restoring by default via env


## 0.1) Input Download and Selection

In [ ]:
!pdb_fetch '1B39' | pdb_selchain -A | pdb_delhetatm > '1B39_A_nohet.pdb'

## 1) Input Preparation

In [ ]:
_, prepared_protein_pdb = client.prepare_protein(
    Path.cwd() / "1B39_A_nohet.pdb", None, None
)

2024-03-22 16:01:10,083 - rush - INFO - Trying to restore job with tags: ['tutorial-resume-gmx'] and path: github:talo/prepare_protein/947cdbc000031e192153a20a9b4a8fbb12279102#prepare_protein_tengu
2024-03-22 16:01:10,145 - rush - WARNING - Multiple module instances found with the same tags and path
2024-03-22 16:01:10,145 - rush - WARNING - Multiple module instances found with the same tags and path
2024-03-22 16:01:10,146 - rush - WARNING - Multiple module instances found with the same tags and path
2024-03-22 16:01:10,146 - rush - WARNING - Multiple module instances found with the same tags and path
2024-03-22 16:01:10,146 - rush - WARNING - Multiple module instances found with the same tags and path
2024-03-22 16:01:10,191 - rush - INFO - Restoring job from previous run with id 3e4e10dd-f7cf-4e33-b759-cc73d6ae11c9


## 2.1) Run GROMACS (modules: gmx, gmx_resume)
Next we will run a molecular dynamics simulation on our protein using gromacs via the `gmx` module.

We'll set `timeout_duration_mins = 1` so that the run times out before it finishes, and then resume via the `gmx_resume` module using the first output, which is the archive that contains all the necessary data for resuming the run from the last saved checkpoint.

We'll set `checkpoint_interval_mins = 1.0/60` so that the checkpointing takes place once per second.

We can restart as many times as we need to in order for the run to finish. Use a unique tag for each restarted run so that each sequential restart will be tagged and cached appropriately. See below for an example.

For each restarted run, please pass the same config file that was passed to the original `gmx` call. There is no support for shortening or extending runs, or changing any other run parameters, of runs that have already been started. Passing the same config ensures that progress is reported properly and that there are no other inconsistencies. So, the initial run config should specify the full desired run.

One current limitation is that the frame selection can only operate on the data generated by the last call to `gmx` or `gmx_restart`. Otherwise, the output xtc files from all the calls must be joined and processed manually.

In [ ]:
gmx_config = {
    "params_overrides": {
        "nvt": {"nsteps": 2000},
        "npt": {"nsteps": 2000},
        "md": {"nsteps": 150000},
    },
    "frame_sel": {
        "start_time_ps": 290,
        "end_time_ps": 300,
        "delta_time_ps": 1,
    },
    "checkpoint_interval_mins": 1.0 / 60,
    "timeout_duration_mins": 1,
    "num_gpus": 1,
    "save_wets": False,
}

In [ ]:
resume_files_first, streaming_outputs, static_outputs, *rest = client.gmx(
    None,
    prepared_protein_pdb,
    None,
    gmx_config,
    resources={"gpus": 1, "storage": 1, "storage_units": "GB"},
)

2024-03-22 16:01:10,200 - rush - INFO - Trying to restore job with tags: ['tutorial-resume-gmx'] and path: github:talo/tengu-gmx/04cff2931b995c33263dfdb477d7f09c8bbd75a7#gmx_tengu
2024-03-22 16:01:10,261 - rush - INFO - Restoring job from previous run with id f6d416cf-0cde-4bf1-8ccd-44c0e71aa0d9


## Checking progress of GROMACS run
To determine if your GROMACS run is done, or if further runs are required, you can look at the progress output.
How to check this output is demonstrated below. 


This is an example progress event that indicates that the job has not completed, and will require resuming.
`n` is the number of execution steps in the GMX module. If `n` is less than `n_expected`, or `done` is false the run is not yet completed.


`gmx_resume_tengu progress: {
  "n": 600,
  "n_expected": 601,
  "n_max": 601,
  "done": false
}`

We keep running resume until the progress is done

In [ ]:
# Checking
result1 = client.module_instance_blocking(resume_files_first.source)
print(result1.progress)

done = result1.progress.done
resumes = 0
resume_files = resume_files_first
while not done:
    resume_files, _, _, xtc_dry, pdb_dry, _, _ = client.gmx_resume(
        resume_files,
        gmx_config,
        tags=[f"gmx-resume-{resumes}"],
        resources={"gpus": 1, "storage": 1, "storage_units": "GB"},
    )
    # wait for module to finish
    resume_files.get()
    progress = client.module_instance_blocking(resume_files.source).progress
    print(progress)
    done = progress.done
    resumes += 1

n=139100 n_expected=154000 n_max=154000 done=False
2024-03-22 16:23:57,187 - rush - INFO - Trying to restore job with tags: ['gmx-resume-0', 'tutorial-resume-gmx'] and path: github:talo/tengu-gmx/04cff2931b995c33263dfdb477d7f09c8bbd75a7#gmx_resume_tengu
2024-03-22 16:23:57,248 - rush - INFO - Restoring job from previous run with id d2d3e95f-c353-49c4-ac6b-8fb7eb07cce4
n=154000 n_expected=154000 n_max=154000 done=True


We can see that we get around 50k steps a run, so we should launch another 2 resume jobs

# Downloading Results
To download extracted frames and fetch their pdbs, we can do the following

In [ ]:
pdb_dry.download("dry_frames.tar.gz")

PosixPath('/home/machineer/qdx/tutorial-gmx-resume/objects/dry_frames.tar.gz')

In [ ]:
import tarfile

with tarfile.open(client.workspace / "objects" / "dry_frames.tar.gz", "r") as tf:
    selected_frame_pdbs = [
        tf.extractfile(member).read()
        for member in tf
        if "pdb" in member.name and member.isfile()
    ]
    for i, frame in enumerate(selected_frame_pdbs):
        with open(
            client.workspace / "objects" / f"gmx_output_frame_{i}.pdb", "w"
        ) as pf:
            print(frame.decode("utf-8"), file=pf)

In [ ]:
with open(client.workspace / "objects" / "gmx_output_frame_0.pdb", "r") as f:
    print(str.join("", f.readlines()[0:10]))

REMARK    GENERATED BY TRJCONV
TITLE     Protein in water t= 292.00000 step= 146000
REMARK    THIS IS A SIMULATION BOX
CRYST1   87.907   87.907   87.907  60.00  60.00  90.00 P 1           1
MODEL        3
ATOM      1  N   MET     1      51.280  58.500   9.380  1.00  0.00           N
ATOM      2  H1  MET     1      50.490  58.640  10.000  1.00  0.00           H
ATOM      3  H2  MET     1      51.070  57.860   8.640  1.00  0.00           H
ATOM      4  H3  MET     1      52.060  58.170   9.920  1.00  0.00           H
ATOM      5  CA  MET     1      51.490  59.810   8.760  1.00  0.00           C

